# Data Cleaning and Munging Script - City Data Portal

In [154]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd

%matplotlib inline
pd.options.mode.chained_assignment = None
sns.set()

In [155]:
austin_zips = (78701,
78702,
78703,
78704,
78705,
78717,
78719,
78721,
78722,
78723,
78724,
78725,
78726,
78727,
78728,
78729,
78730,
78731,
78732,
78733,
78734,
78735,
78736,
78737,
78739,
78741,
78742,
78744,
78745,
78746,
78747,
78748,
78749,
78750,
78751,
78752,
78753,
78754,
78756,
78757,
78758,
78759,
78617,
78653,
78660)

### Permits Data

In [156]:
permits_data = pd.read_csv("raw_data/Issued_Construction_Permits.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (54,58,59,60,61,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
permits = permits_data.loc[:,('Permit Num', 'Permit Class Mapped', 
                              'Calendar Year Issued', 'Original Zip')]

permits = permits[permits['Calendar Year Issued'] >= 2011]
permits = permits[permits['Original Zip'].isin(austin_zips)]

res_permits = permits[permits['Permit Class Mapped'] == "Residential"].drop('Permit Class Mapped', axis = 1)
com_permits = permits[permits['Permit Class Mapped'] == "Commercial"].drop('Permit Class Mapped', axis = 1)

In [17]:
res_permits_year_count = res_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
res_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
res_permits_year_count['var_type'] = 'res_permit_count'

In [18]:
com_permits_year_count = com_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
com_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
com_permits_year_count['var_type'] = 'commercial_permit_count'

In [19]:
res_permits_year_count.rename(columns = {'count': 'res_permit_count'}, inplace = True)
res = res_permits_year_count.drop('var_type', axis = 1)

In [20]:
com_permits_year_count.rename(columns = {'count': 'com_permit_count'}, inplace = True)
com = com_permits_year_count.drop('var_type', axis = 1)

In [157]:
all_var_df = pd.merge(com, res, on = ('year', 'zip_code'), how = 'outer')
all_var_df['year'] = all_var_df['year'].astype(str)
all_var_df['zip_code'] = all_var_df['zip_code'].astype(str)

### Crime Data

In [22]:
crime_data = pd.read_csv("raw_data/Crime_Reports.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
crime = crime_data.loc[:,('Incident Number', 'Highest Offense Description', 'Occurred Date Time', 'Zip Code')]

In [24]:
conditions = [
    (crime['Highest Offense Description'].isin(['ASSAULT BY CONTACT', 'ASSAULT WITH INJURY', 'ASSAULT W/INJURY-FAM/DATE VIOL'])),
    (crime['Highest Offense Description'].isin(['AUTO-THEFT', 'BURGLARY NON RESIDENCE', 'BURGLARY OF RESIDENCE', 'BURGLARY OF VEHICLE', 'THEFT', 'THEFT BY SHOPLIFTING'])),
    (crime['Highest Offense Description'].isin(['CRIMINAL MISCHIEF', 'CRIMINAL TRESPASS', 'DISTURBANCE - OTHER', 'FAMILY DISTURBANCE', 'HARASSMENT'])),
    (crime['Highest Offense Description'].isin(['CUSTODY ARREST TRAFFIC WARR', 'WARRANT ARREST NON TRAFFIC'])),
    (crime['Highest Offense Description'].isin(['DWI', 'PUBLIC INTOXICATION'])),
    (crime['Highest Offense Description'].isin(['IDENTITY THEFT'])),
    (crime['Highest Offense Description'].isin(['POSS CONTROLLED SUB/NARCOTIC', 'POSS OF DRUG PARAPHERNALIA', 'POSSESSION OF MARIJUANA'])),
    (crime['Highest Offense Description'].isin(['RUNAWAY CHILD'])),
    ]
     
values = ['assault', 'theft', 'criminal_activity', 'warrent_arrest', 
          'public_intoxication', 'identity_theft', 'narcotics', 'runaway_child']

crime['Offense Category'] = np.select(conditions, values)
crime = crime[crime['Offense Category'] != '0'].drop('Highest Offense Description', axis = 1)

In [25]:
crime.loc[:,'Occurred Date Time'] = pd.to_datetime(crime.loc[:,'Occurred Date Time'])
crime['year'] = crime.loc[:,'Occurred Date Time'].dt.to_period("Y")
crime = crime.drop('Occurred Date Time', axis = 1)

In [72]:
crime_count = crime.groupby(['Offense Category', 'year', 'Zip Code']).size().reset_index()
crime_count['var_type'] = 'count_' + crime_count['Offense Category'] + '_crime'
crime_count.rename(columns = {'Zip Code': 'zip_code', 0:'count'}, inplace = True)
crime_count = crime_count.drop('Offense Category', axis = 1)
crime_count = crime_count[crime_count['year'] >= pd.Period('2011')]
crime_count = crime_count[crime_count['zip_code'].isin(austin_zips)]

In [73]:
crime_pivot = crime_count.pivot(index=["year", "zip_code"], columns="var_type").reset_index()

In [112]:
crime_join = pd.DataFrame(crime_pivot.loc[:, 'year'])
crime_join['zip_code'] = pd.DataFrame(crime_pivot.loc[:,'zip_code'])
crime_join['count_assault_crime'] = crime_pivot.xs('count_assault_crime', level = 'var_type', axis = 1) 

In [113]:
for col in crime_pivot.columns:
    col_name = col[1]
    if col_name != '':
        crime_join[col_name] = crime_pivot.xs(col_name, level = 'var_type', axis = 1)

In [114]:
for year in range(2011,2021):
    zips = crime_join[crime_join['year'] == str(year)]['zip_code'].unique()
    missing_zips = {element for element in austin_zips if element not in zips}
    print(year, missing_zips)
    for zip_code in missing_zips:
        print(year, zip_code)
       # crime_join = crime_join.append({'year': pd.Period(year), 'zip_code':zip_code, 
            #        'count_assault_crime':0, 'count_criminal_activity_crime':0,
           #       'count_identity_theft_crime': 0, 'count_narcotics_crime': 0, 'count_public_intoxication_crime':0,
          #        'count_runaway_child_crime':0, 'count_theft_crime':0, 'count_warrent_arrest_crime':0}, ignore_index = True)

2011 set()
2012 set()
2013 set()
2014 set()
2015 set()
2016 set()
2017 set()
2018 set()
2019 {78734}
2019 78734
2020 {78734}
2020 78734


In [ ]:
# replace NAs with 0

In [133]:
crime_join['zip_code'] = crime_join['zip_code'].astype(str)
crime_join['year'] = crime_join['year'].astype(str)

In [159]:
all_var_df = pd.merge(all_var_df, crime_join, on = ('year', 'zip_code'), how = 'outer') 

### Final Dataframe

In [163]:
zip_bounds = gpd.read_file("raw_data/austin_area_zip_codes.geojson")

In [167]:
zip_bounds.crs = "EPSG:4326"
zip_bounds['area'] = zip_bounds.to_crs('EPSG:3857').area / (10**6)
zip_bounds

,geodb_oid,objectid,zipcodes_i,zipcode,name,created_by,created_da,modified_b,modified_d,shape_area,shape_len,cartodb_id,created_at,updated_at,geometry,area
0,3,3,56,78739,AUSTIN,None,None,None,None,3.383405e+08,97785.793334,3,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.89002 30.20941, -97.88929 ...",42.205399
1,4,4,57,78645,LEANDER,None,None,None,None,1.043396e+09,196654.442800,4,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.91075 30.50110, -97.91053 ...",130.876693
2,16,16,10,78602,BASTROP,None,None,None,None,5.914482e+09,506803.883037,9,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.27371 30.29662, -97.27323 ...",737.002007
3,17,17,11,78655,MARTINDALE,None,None,None,None,1.846886e+09,212904.793003,10,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.73690 29.78761, -97.73582 ...",228.560595
4,18,18,12,78617,DEL VALLE,None,None,None,None,1.885224e+09,273994.824301,11,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.51624 30.21906, -97.51600 ...",235.007060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,80,80,70,78660,PFLUGERVILLE,None,None,None,None,1.213141e+09,190996.848040,71,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.57653 30.50086, -97.57450 ...",152.155964
76,63,63,74,78662,RED ROCK,None,None,None,None,1.357638e+09,174824.670167,74,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.45105 30.01521, -97.44517 ...",168.567170
77,64,64,75,78705,AUSTIN,None,None,None,None,5.548433e+07,51538.164415,75,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.72978 30.29871, -97.72893 ...",6.937295
78,68,68,79,78701,AUSTIN,None,None,None,None,4.756859e+07,34474.871995,79,2015-06-07T00:48:14+00:00,2015-06-07T00:48:14+00:00,"MULTIPOLYGON (((-97.73599 30.25101, -97.73589 ...",5.944796


In [148]:
all_var_df = all_var_df[(all_var_df['year'] != '2021)' & (all_var_df['year'] != '2020')]
all_var_df['zip_code'] = all_var_df['zip_code'].str[0:5]

In [149]:
all_var_df.to_csv("clean_data/city_portal_data.csv")

## Data - Not Using

### Water Consumption Data

In [135]:
water_grouped = pd.read_csv("raw_data/water_grouped_sum.csv")

In [136]:
water_grouped.rename(columns = {'postal_code': 'zip_code', 'commercial': 'total_gal_water_com', 'residential': 'total_gal_water_res'}, inplace = True)

In [137]:
water_grouped['zip_code'] = water_grouped['zip_code'].astype(float)
water_grouped['zip_code'] = water_grouped['zip_code'].astype(str)
water_grouped['year'] = water_grouped['year'].astype(str)

In [138]:
all_var_df = pd.merge(all_var_df, water_grouped, on = ('year', 'zip_code'), how = 'outer')

### 311 Data

In [139]:
three11_data = pd.read_csv("raw_data/311_Unified_Data_Test_Site_2019.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [141]:
conditions = [
    (three11_data['SR Description'].isin(['Animal - Proper Care', 'Animal Control - Assistance Request', 'Found Animal Report - Keep', 'Injured / Sick Animal','Loose Dog','Wildlife Exposure'])),
    (three11_data['SR Description'].isin(['Austin Code - Request Code Officer'])),
    (three11_data['SR Description'].isin(['Graffiti Abatement', 'Public Health - Graffiti Abatement'])),
    (three11_data['SR Description'].isin(['Loud Commercial Music'])),
    (three11_data['SR Description'].isin(['Park Maintenance - Grounds'])),
    (three11_data['SR Description'].isin(['Debris in Street', 'Pothole Repair', 'Sign - Traffic Sign Maintenance', 'Street Light Issue- Address', 'Traffic Signal - Dig Tess Request', 'Traffic Signal - Maintenance'])),
    ]
     
values = ['animal_311', 'request_code_officer_311', 'graffiti_311',
          'loud_commercial_music_311', 'park_maintenance_311', 'road_repair_311']

three11_data['Reporting Category'] = np.select(conditions, values)
three11_small = three11_data[three11_data['Reporting Category'] != '0'].drop('SR Description', axis = 1)

In [142]:
three11_small.loc[:,'Created Date'] = pd.to_datetime(three11_small.loc[:,'Created Date'])
three11_small['year'] = three11_small.loc[:,'Created Date'].dt.to_period("Y")

In [143]:
three11 = three11_small[three11_small['year'] >= pd.Period(2011)]
three11 = three11[three11['Zip Code'].isin(austin_zips)]

In [144]:
three11_gb = three11.groupby(['Reporting Category', 'year', 'Zip Code']).size().reset_index()
three11_gb['var_type'] = 'count_' + three11_gb['Reporting Category'] + '_report'
three11_gb.rename(columns = {'Zip Code': 'zip_code', 0:'count'}, inplace = True)
three11_gb = three11_gb.drop('Reporting Category', axis = 1)
three11_gb = three11_gb[three11_gb['zip_code'].isin(austin_zips)]

In [145]:
three11_gb['zip_code'] = three11_gb['zip_code'].astype(str)
three11_gb['year'] = three11_gb['year'].astype(str)

In [146]:
three11_pivot = three11_gb.pivot(index=["year", "zip_code"], columns="var_type").reset_index()
three11_join = pd.DataFrame(three11_pivot.loc[:, 'year'])
three11_join['zip_code'] = pd.DataFrame(three11_pivot.loc[:,'zip_code'])

for col in three11_pivot.columns:
    col_name = col[1]
    if col_name != '':
        three11_join[col_name] = three11_pivot.xs(col_name, level = 'var_type', axis = 1)

In [147]:
all_var_df = pd.merge(all_var_df, three11_join, on = ('year', 'zip_code'), how = 'outer')

### Zillow Data

In [44]:
home_val_data = pd.read_csv("raw_data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv")

In [47]:
home_val = home_val_data[home_val_data['CountyName'] == 'Travis County']
home_val

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
12,92593,12,78660,Zip,TX,TX,Pflugerville,Austin-Round Rock,Travis County,153358.0,...,297776.0,300716.0,304199.0,308782.0,313882.0,320012.0,325914.0,331657.0,337919.0,345907.0
69,92617,71,78704,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,NaN,...,693336.0,700205.0,708350.0,718003.0,729798.0,743912.0,759432.0,774389.0,789514.0,805352.0
91,92654,93,78745,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,NaN,...,386880.0,391274.0,396339.0,402291.0,409867.0,418620.0,427082.0,434363.0,442531.0,453053.0
441,92657,446,78748,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,180595.0,...,347213.0,350689.0,355000.0,360434.0,366663.0,373765.0,380892.0,387678.0,395099.0,404343.0
448,92667,453,78758,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,160273.0,...,363830.0,368430.0,374008.0,380226.0,387386.0,395143.0,401828.0,407787.0,414384.0,423721.0
483,92651,488,78741,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,145569.0,...,338322.0,341162.0,344385.0,348804.0,354467.0,360563.0,366086.0,370801.0,376671.0,384309.0
518,92662,523,78753,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,128803.0,...,310840.0,313783.0,317501.0,322012.0,327270.0,333272.0,338868.0,344512.0,350877.0,359218.0
591,92668,597,78759,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,244940.0,...,546330.0,550546.0,556137.0,563533.0,572424.0,582608.0,593182.0,603011.0,613881.0,626430.0
1602,92653,1617,78744,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,112635.0,...,276055.0,279095.0,282623.0,287047.0,292194.0,297844.0,303529.0,308633.0,314300.0,320990.0
2059,92634,2075,78723,Zip,TX,TX,Austin,Austin-Round Rock,Travis County,NaN,...,447712.0,453269.0,458320.0,464157.0,471388.0,480590.0,489950.0,497413.0,505253.0,514774.0


In [49]:
rent_data = pd.read_csv("raw_data/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv")

In [51]:
rent = rent_data[rent_data['MsaName'] == "Austin, TX"]
rent

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03
11,92593,78660,13,"Austin, TX",1261.0,1266.0,1270.0,1274.0,1278.0,1282.0,...,1524.0,1527.0,1529.0,1532.0,1534.0,1537.0,1539.0,1542.0,1545.0,1548.0
29,92551,78613,35,"Austin, TX",1257.0,1262.0,1267.0,1272.0,1277.0,1282.0,...,1499.0,1500.0,1501.0,1502.0,1502.0,1503.0,1504.0,1505.0,1506.0,1507.0
59,92617,78704,72,"Austin, TX",1388.0,1402.0,1417.0,1432.0,1446.0,1461.0,...,1802.0,1797.0,1793.0,1789.0,1785.0,1781.0,1778.0,1774.0,1771.0,1768.0
64,92598,78666,78,"Austin, TX",NaN,NaN,1097.0,NaN,1078.0,1069.0,...,1240.0,1247.0,1254.0,1261.0,1268.0,1275.0,1283.0,1290.0,1298.0,1306.0
74,92654,78745,94,"Austin, TX",1006.0,1018.0,1030.0,1042.0,1053.0,1065.0,...,1374.0,1375.0,1376.0,1377.0,1378.0,1379.0,1380.0,1380.0,1382.0,1383.0
97,92576,78641,125,"Austin, TX",1291.0,1295.0,1300.0,1304.0,1308.0,1313.0,...,1553.0,1560.0,1567.0,1574.0,1582.0,1589.0,1596.0,1604.0,1611.0,1619.0
299,92657,78748,440,"Austin, TX",1006.0,1017.0,NaN,1040.0,1051.0,1062.0,...,1348.0,1350.0,1352.0,1354.0,1356.0,1358.0,1360.0,1362.0,1364.0,1367.0
305,92667,78758,447,"Austin, TX",1042.0,1055.0,1068.0,1082.0,1094.0,1107.0,...,1489.0,1483.0,1477.0,1471.0,1465.0,1459.0,1453.0,1447.0,1441.0,1435.0
331,92651,78741,479,"Austin, TX",NaN,NaN,NaN,1124.0,1130.0,1136.0,...,1393.0,1392.0,1392.0,1392.0,1392.0,1393.0,1393.0,1394.0,1395.0,1395.0
339,92597,78664,494,"Austin, TX",1193.0,1201.0,1209.0,1217.0,1224.0,1232.0,...,1542.0,1546.0,1552.0,1557.0,1563.0,1569.0,1575.0,1580.0,1586.0,1593.0
